In [21]:
import folium
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [22]:
fmap = folium.Map(location=[37.2633325, 127.0287472], tiles="Stamen Toner", zoom_start=17, width=800, height=600)
# tiles="cartodbpositron"

folium.Marker([37.2633325, 127.0287472], tooltip="Suwon City Hall", icon=folium.Icon(color='red', icon='star')).add_to(fmap)

In [23]:
"""
Randomly generated geopoint (http://www.geomidpoint.com/random/)
	# of points : 100
	starting point : Suwon City Hall
	within 1km distance (circular region)
"""
data = pd.read_csv("./data/random.csv", usecols=['latitude', 'longitude'])

d_list = data.values.tolist()
print(d_list[0])

[37.26406042, 127.02466555]


In [25]:
fire_center = d_list[len(d_list) - 1]
print(fire_center)
folium.Circle(fire_center, tooltip='fire', radius=300, color='red', fill_color='red').add_to(fmap)
# radius in meters

for i in range (0, len(d_list) - 1):
	folium.Circle(d_list[i], tooltip=str(i), radius=2).add_to(fmap)

fmap

[37.26549784, 127.03400625]


In [ ]:
def haversine_distance(point, center):
	del_lat = point[0] - center[0]
	del_lon = point[1] - center[1]

	a = sin(del_lat / 2) ** 2 + cos(point[0]) * cos(center[0]) * sin(del_lon / 2) **2
	c = 2 * asin(sqrt(a))
	r = 6371 # radius of earth in km
	return c * r * 1000 # return in m

In [ ]:
def get_inside(d_list, center, radius):
	return